In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

Regresion Lineal:

In [23]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [24]:
# Cargo el set de datos diabetes
diabetes = datasets.load_diabetes() #cargo el set de datos
df = pd.DataFrame(diabetes.data, columns = diabetes.feature_names) #paso a un df los datos(x) y pongo en los nombres de los atributos en las columnas 
df['disease progression'] = diabetes.target #la agrego la columna de lo que quiero predecir, "targets"
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,disease progression
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [25]:
#Uso todos los datos y los separa train and test set.
diabetes_X = diabetes.data
diabetes_Y = diabetes.target
seed = 104 #semilla para el random
test_size=0.2#(el tamaño del test es el 20% de los datos)
x_train, x_test, y_train, y_test = train_test_split(diabetes_X, diabetes_Y, test_size = test_size, random_state = seed)
#Creo el objeto de RL y luego uso su metodo de fiteo sobre los X de train
rl = linear_model.LinearRegression()
rl.fit(x_train, y_train)
#Obtencion de resultados
diabetes_pred = rl.predict(x_test) #prediccion por el modelo
print('The coefficients are: \n {0}\n\n'.format(rl.coef_)) #imprime los coeficientes
print('The mean squared error is: {0}\n'.format(mean_squared_error(y_test, diabetes_pred))) #imprime el error cuadratico
print('The variance score is : {0}\n'.format(r2_score(y_test, diabetes_pred))) #imprime la varianza

The coefficients are: 
 [ -17.43992764 -222.28076153  565.50735481  306.2871376  -781.06326464
  483.36565804  138.13843392  191.52513149  728.52458089   45.79289919]


The mean squared error is: 2623.5316218513085

The variance score is : 0.5620438671817145



Ajustar una regresión lineal que modele la propina en función del precio de la comida, utilizando todos los datos.(con tips.csv)

In [60]:
df_tips = pd.read_csv('tips.csv')
df_tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [54]:
#primero tengo que convertir las varibles que son string en numericas:
#obs:esta es la explicacion para una sola columna, pero mas abajo hay un general
from sklearn.preprocessing import LabelEncoder
# con esta funcion puedo ver si alguna columna son string --> isinstance(df_titanic.Sex[0],str)
le = LabelEncoder() #creo objeto
le.fit(df_tips.sex) #transformo sex en numerico
sex_encoded = le.transform(df_tips.sex) #guardo la columna sex en una variable
df_tips.sex = sex_encoded # cambio la antigua columna por la nueva
labels_sex=le.classes_ #guardo en una variable los lables y como fueron modificados:
print(labels_sex) #como aparece primero Female y despues male ==> Female=0 y Male=1
df_tips.head()

['Female' 'Male']


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0,No,Sun,Dinner,2
1,10.34,1.66,1,No,Sun,Dinner,3
2,21.01,3.50,1,No,Sun,Dinner,3
3,23.68,3.31,1,No,Sun,Dinner,2
4,24.59,3.61,0,No,Sun,Dinner,4


In [61]:
#idem para los otros que faltan
le = LabelEncoder() #creo objeto
labels=list()
for i in df_tips.columns:
    if(isinstance(df_tips[i][0],str)==True):
        le.fit(df_tips[i]) #transformo sex en numerico
        encoded = le.transform(df_tips[i]) #guardo la columna sex en una variable
        df_tips[i] = encoded # cambio la antigua columna por la nueva
        labels.append(le.classes_) #guardo en una variable los lables y como fueron modificados:

In [62]:
print(labels)
df_tips.head()

[array(['Female', 'Male'], dtype=object), array(['No', 'Yes'], dtype=object), array(['Fri', 'Sat', 'Sun', 'Thur'], dtype=object), array(['Dinner', 'Lunch'], dtype=object)]


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0,0,2,0,2
1,10.34,1.66,1,0,2,0,3
2,21.01,3.50,1,0,2,0,3
3,23.68,3.31,1,0,2,0,2
4,24.59,3.61,0,0,2,0,4


In [19]:
#otra forma de hacerlo, pero perdes los labels
columnas_no_numericas=["sex","smoker","day","time"]
for col in (df_tips[columnas_no_numericas]).columns:
    df_tips[col] = pd.Categorical(df_tips[col] ).codes
X = df_tips[["total_bill","sex","smoker","day","time","size"]]
Y = df_tips.tip

In [93]:
seed = 104 #semilla para el random
test_size=0.2#(el tamaño del test es el 20% de los datos)
xtrain, xtest, ytrain, ytest = train_test_split(df_tips.drop(["tip"],axis=1), df_tips['tip'], test_size = 0.2, random_state = seed)
rl = linear_model.LinearRegression()
rl.fit(xtrain, ytrain)
preds = rl.predict(xtest)
print('The coefficients are: \n {0}\n\n'.format(rl.coef_))
print('The mean squared error is: {0}\n'.format(mean_squared_error(ytest, preds)))
print('The variance score is : {0}\n'.format(r2_score(ytest, preds))) 
rl.score(xtest,ytest)

The coefficients are: 
 [ 0.09219328 -0.07445596 -0.06438951 -0.03282318  0.15072696  0.20018295]


The mean squared error is: 1.448375407010042

The variance score is : 0.4397424259436816



0.4397424259436816

Utilizar  la  regresi  ́on  anterior  para  enstimar  la  propina  que  deja  unamujer  no  fumadora  si  la  cuenta  es  de  pesos  $20.

In [98]:
#"total_bill","sex","smoker","day","time","size"
total_bill=20
sex=0
smoker=0
day = 2 # lunes
time = 0 #cena
size = 1

x_test_aux = np.array([total_bill, sex, smoker, day, time, size]).reshape(1, -1)
pred=rl.predict(x_test_aux)
print('La propina que dejó la mujer fue: {0}'.format(pred))

La propina que dejó la mujer fue: [2.72816764]
